In [159]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0'],axis=1)

tr = test.loc[test.x_exit.isnull()]

def conv(time_1):   
    sec1 = 3600*int(time_1[0:2]) + 60*int(time_1[3:5]) + int(time_1[6:8])                                                                                                                                
    return (sec1)

# vec_conv =  np.vectorize(conv)
# train.time_entry=vec_conv(train.time_entry)
# train.time_exit=vec_conv(train.time_exit)

#incohérences
train['vmean'][(train.time_entry==train.time_exit)] = 0
train['vmin'][(train.time_entry==train.time_exit)] = 0
train['vmax'][(train.time_entry==train.time_exit)] = 0

test['vmean'][(test.time_entry==test.time_exit)] = 0
test['vmin'][(test.time_entry==test.time_exit)] = 0
test['vmax'][(test.time_entry==test.time_exit)] = 0

train['vmax'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmin'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmean'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0

test['vmax'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmin'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmean'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0

#outliers
train.vmean = train.vmean.loc[(train.vmean<90) & (train.vmean>=0)]
train.vmax = train.vmean.loc[(train.vmax<90) & (train.vmax>=0)]
train.vmin = train.vmean.loc[(train.vmin<90) & (train.vmin>=0)]

#nan by mean
train['vmax']=train['vmax'].replace( np.NaN , train['vmax'].mean() )
train['vmin']=train['vmin'].replace( np.NaN , train['vmin'].mean() )
train['vmean']=train['vmean'].replace( np.NaN , train['vmean'].mean() )

test['vmax']=test['vmax'].replace( np.NaN , test['vmax'].mean() )
test['vmin']=test['vmin'].replace( np.NaN , test['vmin'].mean() )
test['vmean']=test['vmean'].replace( np.NaN , test['vmean'].mean() )

vec_conv =  np.vectorize(conv)
train.time_entry=vec_conv(train.time_entry)
train.time_exit=vec_conv(train.time_exit)

test.time_entry=vec_conv(test.time_entry)
test.time_exit=vec_conv(test.time_exit)

C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [160]:
train['city_center'] = 0
train['city_center'][(train.x_exit>=3750901.5068) & (train.x_exit<=3770901.5068)&(train.y_exit>=(-19268905.6133)) & (train.y_exit<=(-19208905.6133))]=1

C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [161]:
train = train.drop(['trajectory_id'],axis=1)
test = test.drop(['trajectory_id'],axis=1)

In [162]:
a = train.values
c = test.values

In [163]:
b=[]
i=-1
hash = ''
for line in a:
    if line[0] != hash:
        if i>=0:
            b[i] = np.concatenate((b[i][:9],b[i][11:]),axis=None)
            #enlève x_ et y_ du dernier trajet 

        hash = line[0]
        ref = line[1:-1]
        b.append(np.concatenate((line[-1],hash,ref),axis=None))
        i=i+1
    else:
        ref = np.concatenate((line[1:-1],ref), axis=None)
        b[i]=np.concatenate((line[-1],hash,ref),axis=None)

In [164]:
df_train = pd.DataFrame(b)
Y=df_train[0]
X = df_train.drop([0,1],axis=1)
X=X.fillna(0)

In [165]:
b=[]
i=-1
hash = ''
for line in c:
    if line[0] != hash:
        if i>=0:
            b[i] = np.concatenate((b[i][:8],b[i][10:]),axis=None)
            #enlève x_ et y_ du dernier trajet dans le test cest NaN mais on doit l'enlever quand même
        hash = line[0]
        ref = line[1:]
        b.append(np.concatenate((hash,ref),axis=None))
        i=i+1
    else:
        ref = np.concatenate((line[1:],ref), axis=None)
        b[i]=np.concatenate((hash,ref),axis=None)

In [166]:
df_test = pd.DataFrame(b)

X_test = df_test.drop([0],axis=1)
X_test=X_test.fillna(0)

In [167]:
#df_test.to_csv('C:/Users/gouzm/Documents/code/DataScience/Challenge EY/df_test.csv')

In [168]:
#df_train.to_csv('C:/Users/gouzm/Documents/code/DataScience/Challenge EY/df_train.csv')

In [169]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
0,00032f51796fd5437b238e3a9823d13d_31,54212,54632,1.021057,0.851465,0.959013,3.773118e+06,-1.914490e+07,48333.0,49393.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000479418b5561ab694a2870cc04fd43_25,55749,55856,1.021057,0.851465,0.959013,3.769978e+06,-1.934136e+07,43703.0,44004.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000506a39775e5bca661ac80e3f466eb_29,55568,55568,0.000000,0.000000,0.000000,3.757468e+06,-1.923860e+07,40142.0,40142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0005401ceddaf27a9b7f0d42ef1fbe95_1,56118,56525,1.021057,0.851465,0.959013,3.760505e+06,-1.935500e+07,34659.0,34659.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00063a4f6c12e1e4de7d876580620667_3,53647,54314,1.021057,0.851465,0.959013,3.766319e+06,-1.917013e+07,51500.0,52502.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
#clf = XGBClassifier(random_state=0,n_jobs=-1)

In [171]:
#clf.fit(X.values,Y.values)

In [172]:
X = X
y = Y
param_grid = {
    'num_boost_round': [100,300],
    'eta': [0.1,0.01],
    'max_depth': [5,6,8],
    'min_child_weight': [0.5,1,2],
    'n_jobs':[-1]
}

i=1
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True)

for train_index,test_index in kf.split(X,y):
    print('\n{} of kfold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y[train_index],y[test_index]
    model = GridSearchCV(XGBClassifier(), param_grid, cv=5, scoring= 'f1',iid=True)
    print ('---')
    model.fit(xtr, ytr)
    print (model.best_params_)
    pred=model.predict(xvl)
    print('accuracy_score',accuracy_score(yvl,pred))
    i+=1


1 of kfold 5
---
{'eta': 0.1, 'max_depth': 8, 'min_child_weight': 0.5, 'n_jobs': -1, 'num_boost_round': 100}
accuracy_score 0.9383507999850819

2 of kfold 5
---
{'eta': 0.1, 'max_depth': 8, 'min_child_weight': 2, 'n_jobs': -1, 'num_boost_round': 100}
accuracy_score 0.9368216909707977

3 of kfold 5
---
{'eta': 0.1, 'max_depth': 8, 'min_child_weight': 2, 'n_jobs': -1, 'num_boost_round': 100}
accuracy_score 0.9364114422108679

4 of kfold 5
---
{'eta': 0.1, 'max_depth': 8, 'min_child_weight': 1, 'n_jobs': -1, 'num_boost_round': 100}
accuracy_score 0.9394301059227211

5 of kfold 5
---
{'eta': 0.1, 'max_depth': 8, 'min_child_weight': 0.5, 'n_jobs': -1, 'num_boost_round': 100}
accuracy_score 0.9347307175891392


In [174]:
pred = model.predict(X_test)
prediction = pd.DataFrame()

ValueError: feature_names mismatch: ['2  ', '3  ', '4  ', '5  ', '6  ', '7  ', '8  ', '9  ', '10 ', '11 ', '12 ', '13 ', '14 ', '15 ', '16 ', '17 ', '18 ', '19 ', '20 ', '21 ', '22 ', '23 ', '24 ', '25 ', '26 ', '27 ', '28 ', '29 ', '30 ', '31 ', '32 ', '33 ', '34 ', '35 ', '36 ', '37 ', '38 ', '39 ', '40 ', '41 ', '42 ', '43 ', '44 ', '45 ', '46 ', '47 ', '48 ', '49 ', '50 ', '51 ', '52 ', '53 ', '54 ', '55 ', '56 ', '57 ', '58 ', '59 ', '60 ', '61 ', '62 ', '63 ', '64 ', '65 ', '66 ', '67 ', '68 ', '69 ', '70 ', '71 ', '72 ', '73 ', '74 ', '75 ', '76 ', '77 ', '78 ', '79 ', '80 ', '81 ', '82 ', '83 ', '84 ', '85 ', '86 ', '87 ', '88 ', '89 ', '90 ', '91 ', '92 ', '93 ', '94 ', '95 ', '96 ', '97 ', '98 ', '99 ', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139

In [ ]:
trajectory_id = tr['trajectory_id']

In [ ]:
trajectory_id=trajectory_id.values
prediction['id']=trajectory_id
prediction['target']=pred

In [ ]:
prediction.to_csv("EYChallenge_All_in_one_line_best.csv",index=False)

In [175]:
print(model.best_params_)

{'eta': 0.1, 'max_depth': 8, 'min_child_weight': 0.5, 'n_jobs': -1, 'num_boost_round': 100}
